In [27]:
import pandas as pd
import numpy as np
import mlflow
import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score, f1_score
import lightgbm as lgb
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [3]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("my_first_try")

2022/05/31 11:59:53 INFO mlflow.tracking.fluent: Experiment with name 'my_first_try' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/3', experiment_id='3', lifecycle_stage='active', name='my_first_try', tags={}>

In [7]:
wine = load_wine()
X, y = wine.data, wine.target

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val =  train_test_split(X_train, y_train, test_size=0.2)

In [41]:
regressor = lgb.LGBMClassifier()
param_grid = {
    'n_estimators': [400, 700, 1000],
    'colsample_bytree': [0.7, 0.8],
    'max_depth': [15,20,25],
    'num_leaves': [50, 100, 200],
    'reg_alpha': [1.1, 1.2, 1.3],
    'reg_lambda': [1.1, 1.2, 1.3],
    'min_split_gain': [0.3, 0.4],
    'subsample': [0.7, 0.8, 0.9],
    'subsample_freq': [20]
}
cv = KFold(n_splits=5, shuffle=True, random_state=42).split(X=X_train, y=y_train)


In [51]:
with mlflow.start_run():
    mlflow.lightgbm.autolog()

    learning_rate=0.1

    classifier = lgb.LGBMClassifier(learning_rate = learning_rate)
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    
    accuracy = accuracy_score(y_pred, y_test)
    f1 = f1_score(y_pred, y_test, average = None)
    
    mlflow.log_metric("accuracy", accuracy)

    mlflow.lightgbm.log_model(classifier, artifact_path="models_mlflow")


In [52]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)



In [54]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./artifacts/1', experiment_id='1', lifecycle_stage='active', name='random-forest-hyperopt', tags={}>,
 <Experiment: artifact_location='./artifacts/2', experiment_id='2', lifecycle_stage='active', name='random-forest-best-models', tags={}>,
 <Experiment: artifact_location='./mlruns/3', experiment_id='3', lifecycle_stage='active', name='my_first_try', tags={}>]

In [56]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids = "3",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5
)

In [58]:
mlflow.register_model(model_uri="6628510cfc7d45908f5966c763e592ef", name = "lightgbm_model")

Registered model 'lightgbm_model' already exists. Creating a new version of this model...
2022/05/31 15:20:32 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: lightgbm_model, version 2
Created version '2' of model 'lightgbm_model'.


<ModelVersion: creation_timestamp=1654003231998, current_stage='None', description=None, last_updated_timestamp=1654003231998, name='lightgbm_model', run_id=None, run_link=None, source='6628510cfc7d45908f5966c763e592ef', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [61]:
client.get_latest_versions(name = "lightgbm_model")

[<ModelVersion: creation_timestamp=1654003231998, current_stage='None', description=None, last_updated_timestamp=1654003231998, name='lightgbm_model', run_id=None, run_link=None, source='6628510cfc7d45908f5966c763e592ef', status='READY', status_message=None, tags={}, user_id=None, version=2>]

In [69]:
from datetime import datetime

date = datetime.today()
new_stage = "Production"
version = 2
client.transition_model_version_stage(
    name = "lightgbm_model",
    version = version,
    stage = new_stage
)

<ModelVersion: creation_timestamp=1654003231998, current_stage='Production', description='The model version 2 was transitioned to Staging on 2022-05-31 15:31:44.209676', last_updated_timestamp=1654004281270, name='lightgbm_model', run_id=None, run_link=None, source='6628510cfc7d45908f5966c763e592ef', status='READY', status_message=None, tags={}, user_id=None, version=2>